<a href="https://colab.research.google.com/github/akim327/cis700-text-adventures-hw2/blob/master/my_game_improved.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text adventure game

This Python notebook builds a simple text advenutre game inspired by the [Adventuron Classroom](https://adventuron.io/classroom/) design by Chris Ainsley of Adventuron Software Limited.

The main components are:
1. __The parser__, which interprets the player's commands.
2. __The game__, which represents the world (a collection of __locations__ and __items__), and describes what the player sees.
3. __The data__, which you input to create your own unique game.

## The Game Class
The game keeps track of the state of the world, and describes what the player sees as they move through different locations.

In [0]:
class Game:
  """The Game class represents the world.  Internally, we use a 
     graph of Location objects and Item objects, which can be at a 
     Location or in the player's inventory.  Each locations has a set of
     exits which are the directions that a player can move to get to an
     adjacent location. The player can move from one location to another
     location by typing a command like "Go North".
  """

  def __init__(self, start_at):
    # start_at is the location in the game where the player starts
    self.curr_location = start_at
    self.curr_location.has_been_visited = True
    # inventory is the set of objects that the player has collected/
    self.inventory = {}
    # equipment is the set of objects that the player is wearing
    self.equipment = {}
    # Print the special commands associated with items in the game (helpful 
    # for debugging and for novice players).
    self.print_commands = True

  def describe(self):
    """Describe the current game state by first describing the current 
       location, then listing any exits, and then describing any objects
       in the current location."""
    self.describe_current_location()
    self.describe_exits()
    self.describe_items()

  def describe_current_location(self):
    """Describe the current location by printing its description field."""
    print(self.curr_location.description)

  def describe_exits(self):
    """List the directions that the player can take to exit from the current
       location."""
    exits = []
    for exit in self.curr_location.connections.keys():
      exits.append(exit.capitalize())
    if len(exits) > 0:
      print("Exits: ", end = '')
      print(*exits, sep = ", ",)
  
  def describe_items(self):
    """Describe what objects are in the current location."""
    if len(self.curr_location.items) > 0:
      print("You see: ")
      for item_name in self.curr_location.items:
        item = self.curr_location.items[item_name]
        print(item.description)
        if self.print_commands:
          special_commands = item.get_commands()
          for cmd in special_commands:
            print('\t', cmd)

  def add_to_inventory(self, item):
    """Add an item to the player's inventory."""
    self.inventory[item.name] = item
  
  def is_in_inventory(self,item):
    return item.name in self.inventory
  
  def equip(self, item):
    self.equipment[item.name] = item

  def is_equipped(self, item):
    return item.name in self.equipment

  def get_items_in_scope(self):
    """Returns a list of items in the current location and in the inventory"""
    items_in_scope = []
    for item_name in self.curr_location.items:
      items_in_scope.append(self.curr_location.items[item_name])
    for item_name in self.inventory:
      items_in_scope.append(self.inventory[item_name])
    return items_in_scope

## Locations

Locations Locations are the places in the game that a player can visit.  They contain connects to other locations and items that the player can interact with.

In [0]:
class Location:
  """Locations are the places in the game that a player can visit.
     Internally they are represented nodes in a graph.  Each location stores
     a description of the location, any items in the location, its connections
     to adjacent locations, and any blocks that prevent movement to an adjacent
     location.  The connections is a dictionary whose keys are directions and
     whose values are the location that is the result of traveling in that 
     direction.  The travel_descriptions also has directions as keys, and its 
     values are an optional short desciption of traveling to that location.
  """
  def __init__(self, name, description, end_game=False):
    # A short name for the location
    self.name = name
    # A description of the location
    self.description = description
    # True if entering this location should end the game
    self.end_game = end_game
    # Dictionary mapping from directions to other Location objects
    self.connections = {}
    # Dictionary mapping from directions to text description of the path there
    self.travel_descriptions = {}
    # Dictionary mapping from item name to Item objects present in this location
    self.items = {}
    # Dictionary mapping from direction to Block object in that direction
    self.blocks = {}
    # Flag that gets set to True once this location has been visited by player
    self.has_been_visited = False

  def add_connection(self, direction, connected_location, travel_description=""):
    """Add a connection from the current location to a connected location.
       Direction is a string that the player can use to get to the connected
       location.  If the direction is a cardinal direction, then we also 
       automatically make a connection in the reverse direction."""
    self.connections[direction] = connected_location
    self.travel_descriptions[direction] = travel_description
    if direction == 'north':
      connected_location.connections["south"] = self
      connected_location.travel_descriptions["south"] = ""
    if direction == 'south':
      connected_location.connections["north"] = self
      connected_location.travel_descriptions["north"] = ""
    if direction == 'east':
      connected_location.connections["west"] = self
      connected_location.travel_descriptions["west"] = ""
    if direction == 'west':
      connected_location.connections["east"] = self
      connected_location.travel_descriptions["east"] = ""
    if direction == 'up':
      connected_location.connections["down"] = self
      connected_location.travel_descriptions["down"] = ""
    if direction == 'down':
      connected_location.connections["up"] = self
      connected_location.travel_descriptions["up"] = ""
    if direction == 'in':
      connected_location.connections["out"] = self
      connected_location.travel_descriptions["out"] = ""
    if direction == 'out':
      connected_location.connections["in"] = self
      connected_location.travel_descriptions["in"] = ""


  def add_item(self, name, item):
    """Put an item in this location."""
    self.items[name] = item

  def remove_item(self, item):
    """Remove an item from this location (for instance, if the player picks it
       up and puts it in their inventory)."""
    self.items.pop(item.name)


  def is_blocked(self, direction, game):
    """Check to if there is an obstacle in this direction."""
    if not direction in self.blocks:
        return False
    (block_description, preconditions) = self.blocks[direction]
    if check_preconditions(preconditions, game):
      # All the preconditions have been met.  You may pass.
      return False
    else: 
      # There are still obstalces to overcome or puzzles to solve.
      return True

  def get_block_description(self, direction):
    """Check to if there is an obstacle in this direction."""
    if not direction in self.blocks:
      return ""
    else:
      (block_description, preconditions) = self.blocks[direction]
      return block_description

  def add_block(self, blocked_direction, block_description, preconditions):
    """Create an obstacle that prevents a player from moving in the blocked 
       location until the preconditions are all met."""
    self.blocks[blocked_direction] = (block_description, preconditions)

## Checking Preconditions 
In text adventure games it's common to block a player's progress by creating blocks that prevent them from moving to a location.  For instance, a drawbridge might have a troll that you need to get rig of before you can cross into the castle, or a locked door might prevent you from entering a building until you have a key.  

This is a function that you can modify to include other preconditions.

In [0]:
def check_preconditions(preconditions, game, print_failure_reasons=True):
  """Checks whether the player has met all of the specified preconditions"""
  all_conditions_met = True
  for check in preconditions: 
    if check == "inventory_contains":
      item = preconditions[check]
      if not game.is_in_inventory(item):
        all_conditions_met = False
        # if print_failure_reasons:
        #   print("You don't have the %s" % item.name)
    if check == "in_location":
      location = preconditions[check]
      if not game.curr_location == location:
        all_conditions_met = False
        # if print_failure_reasons:
        #   print("You aren't in the correct location")
    if check == "!location_has_item":
      item = preconditions[check]
      if item.name in game.curr_location.items:
        all_conditions_met = False
        # if print_failure_reasons:
        #   print("The %s isn't in this location" % item.name)
    # todo - add other types of preconditions
    if check == "is_equipped":
      item = preconditions[check]
      if not game.is_equipped(item):
        all_conditions_met = False
  return all_conditions_met

## Items
Items are objects that a player can get, or scenery that a player can examine. We could also implement people as items.  

In [0]:
class Item:
  """Items are objects that a player can get, or scenery that a player can
     examine."""
  def __init__(self,
               name,
               description,
               examine_text="",
               take_text="",
               start_at=None,
               gettable=True,
               end_game=False):
    # The name of the object
    self.name = name
    # The default description of the object.
    self.description = description
    # The detailed description of the player examines the object.
    self.examine_text = examine_text
    # Text that displays when player takes an object.
    self.take_text = take_text if take_text else ("You take the %s." % self.name)
    # Indicates whether a player can get the object and put it in their inventory.
    self.gettable = gettable
    # True if entering this location should end the game.
    self.end_game = end_game
    # The location in the Game where the object starts.
    if start_at:
      start_at.add_item(name, self)
    self.commands = {}


  def get_commands(self):
    """Returns a list of special commands associated with this object"""
    return self.commands.keys()

  def add_action(self, command_text, function, arguments, cant_do_text="", preconditions={}):
    """Add a special action associated with this item"""
    self.commands[command_text] = (function, arguments, cant_do_text, preconditions)

  def do_action(self, command_text, game):
    """Perform a special action associated with this item"""
    end_game = False  # Switches to True if this action ends the game.
    if command_text in self.commands:
      function, arguments, cant_do_text, preconditions = self.commands[command_text]
      if check_preconditions(preconditions, game):
        end_game = function(game, arguments)
      else:
        print(cant_do_text)
    else:
      print("Cannot perform the action %s" % command_text)
    return end_game

## The Parser
The parser is the module that handles the natural language understanding in the game.  The players enter commands in text, and the parser interprets them and performs the actions that the player intends.  This is the module with the most potential for improvement using modern natural language processing.  The implementation that I have given below only uses simple keyword matching.

In [0]:
class Parser:
  """The Parser is the class that handles the player's input.  The player 
     writes commands, and the parser performs natural language understanding
     in order to interpret what the player intended, and how that intent
     is reflected in the simulated world. 
  """
  def __init__(self, game):
    # A list of all of the commands that the player has issued.
    self.command_history = []
    # A pointer to the game.
    self.game = game

  def get_player_intent(self,command):
    command = command.lower()
    if "," in command:
      # Let the player type in a comma separted sequence of commands
      return "sequence"
    elif self.get_direction(command):
      # Check for the direction intent
      return "direction"
    elif command.lower() == "look" or command.lower() == "l":
      # when the user issues a "look" command, re-describe what they see
      return "redescribe"
    elif "examine " in command or command.lower().startswith("x "):
      return "examine"
    elif  "take " in command or "get " in command:
      return "take"
    elif "drop " in command:
      return "drop"
    elif "inventory" in command or command.lower() == "i":
      return "inventory"
    else: 
      for item in self.game.get_items_in_scope():
        special_commands = item.get_commands()
        for special_command in special_commands:
          if command == special_command.lower():
            return "special"

  def parse_command(self, command):
    # add this command to the history
    self.command_history.append(command)

    # By default, none of the intents end the game. The following are ways this
    # flag can be changed to True.
    # * Going to a certain place.
    # * Entering a certain special command
    # * Picking up a certain object.

    end_game = False

    blob = TextBlob(command)

    # Intents are functions that can be executed
    intent = self.get_player_intent(command)
    if intent == "direction":
      end_game = self.go_in_direction(command)
    elif intent == "redescribe":
      self.game.describe()
    elif intent == "examine":
      self.examine(command)
    elif intent == "take":
      end_game = self.take(command)
    elif intent == "drop":
      self.drop(command)
    elif intent == "inventory":
      self.check_inventory(command)
    elif intent == "special":
      end_game = self.run_special_command(command)
    elif intent == "sequence":
      end_game = self.execute_sequence(command)
    elif blob.sentiment.polarity < -.3:
      print("The ship AI does not appreciate your negativity, and has decided to off you. YOU LOSE")
      return True
    else:
      print("I'm not sure what you want to do.")
    return end_game

  ### Intent Functions ###

  def go_in_direction(self, command):
    """ The user wants to in some direction """
    direction = self.get_direction(command)

    if direction:
      if direction in self.game.curr_location.connections:
        if self.game.curr_location.is_blocked(direction, self.game):
          # check to see whether that direction is blocked.
          print(self.game.curr_location.get_block_description(direction))
        else:
          # if it's not blocked, then move there 
          self.game.curr_location = self.game.curr_location.connections[direction]

          # If moving to this location ends the game, only describe the location
          # and not the available items or actions.
          if self.game.curr_location.end_game:
            self.game.describe_current_location()
          else:
            self.game.describe()
      else:
        print("You can't go %s from here." % direction.capitalize())
    return self.game.curr_location.end_game

  def check_inventory(self,command):
    """ The player wants to check their inventory"""
    if len(self.game.inventory) == 0:
      print("You don't have anything.")
    else:
      descriptions = []
      for item_name in self.game.inventory:
        item = self.game.inventory[item_name]
        descriptions.append(item.description)
      print("You have: ", end = '')
      print(*descriptions, sep = ", ",)
  

  def examine(self, command):
    """ The player wants to examine something """
    command = command.lower()
    matched_item = False
    # check whether any of the items at this location match the command
    for item_name in self.game.curr_location.items:
      if item_name in command:
        item = self.game.curr_location.items[item_name]
        if item.examine_text:
          print(item.examine_text)
          matched_item = True
        break
    # check whether any of the items in the inventory match the command
    for item_name in self.game.inventory:
      if item_name in command:
        item = self.game.inventory[item_name]
        if item.examine_text:
          print(item.examine_text)
          matched_item = True
    # fail
    if not matched_item:
      print("You don't see anything special.")


  def take(self, command):
    """ The player wants to put something in their inventory """
    command = command.lower()
    matched_item = False

    # This gets set to True if posession of this object ends the game.
    end_game = False

    # check whether any of the items at this location match the command
    for item_name in self.game.curr_location.items:
      if item_name in command:
        item = self.game.curr_location.items[item_name]
        if item.gettable:
          self.game.add_to_inventory(item)
          self.game.curr_location.remove_item(item)
          print(item.take_text)
          end_game = item.end_game
        else:
          print("You cannot take the %s." % item_name)
        matched_item = True
        break
    # check whether any of the items in the inventory match the command
    if not matched_item:
      for item_name in self.game.inventory:
        if item_name in command:
          print("You already have the %s." % item_name)
          matched_item = True
    # fail
    if not matched_item:
      print("You can't find it.")

    return end_game

  def drop(self, command):
    """ The player wants to remove something from their inventory """
    command = command.lower()
    matched_item = False
    # check whether any of the items in the inventory match the command
    if not matched_item:
      for item_name in self.game.inventory:
        if item_name in command:
          matched_item = True
          item = self.game.inventory[item_name]
          self.game.curr_location.add_item(item_name, item)
          self.game.inventory.pop(item_name)
          print("You drop the %s." % item_name)
          break
    # fail
    if not matched_item:
      print("You don't have that.")


  def run_special_command(self, command):
    """Run a special command associated with one of the items in this location
       or in the player's inventory"""
    for item in self.game.get_items_in_scope():
        special_commands = item.get_commands()
        for special_command in special_commands:
          if command == special_command.lower():
            return item.do_action(special_command, self.game)

  def execute_sequence(self, command):
    for cmd in command.split(","):
      cmd = cmd.strip()
      self.parse_command(cmd)

  def get_direction(self, command):
    command = command.lower()
    if command == "n" or "north" in command:
      return "north" 
    if command == "s" or "south" in command:
      return "south"
    if command == "e" or "east" in command: 
      return "east"
    if command == "w" or "west" in command:
      return "west"
    if command == "up":
      return "up"
    if command == "down":
      return "down"
    if command.startswith("go out"):
      return "out"
    if command.startswith("go in"):
      return "in"
    for exit in self.game.curr_location.connections.keys():
      if command == exit.lower() or command == "go " + exit.lower():
        return exit
    return None

## Special functions
Many times we want to add special behavior to items in the game.  For instance, we might want to be able to _pick a rose_ from a _rosebush_, or the _eat_ a _fish_.  In this implementation we do this in a pretty generic way by allowing the game developer to call ```Item.add_action(cmd,function,argment,preconditions)``` where ```function``` is any Python function. Some example of functions are defined below.

These functions should return True if the game is ended by the action, False otherwise.

In [0]:
def add_item_to_inventory(game, *args):
  """ Add a newly created Item and add it to your inventory."""
  (item, action_description, already_done_description) = args[0]
  if(not game.is_in_inventory(item)):
    print(action_description)
    game.add_to_inventory(item)
  else:
    print(already_done_description)
  return False

def equip_item(game, *args):
  """Equip an item."""
  (item, action_description) = args[0]
  if(game.is_in_inventory(item)):
    print(action_description)
    game.equip(item)
  elif(is_equipped(item)):
    print("It's already equipped.")
  else:
    print("You don't have %s." % item.name)

def describe_something(game, *args):
  """Describe some aspect of the Item"""
  (description) = args[0]
  print(description)
  return False

def destroy_item(game, *args):
  """Removes an Item from the game by setting its location is set to None."""
  (item, action_description, already_done_description) = args[0]
  if game.is_in_inventory(item):
    game.inventory.pop(item.name)
    print(action_description)
  elif item.name in game.curr_location.items:
    game.curr_location.remove_item(item)
    print(action_description)
  else:
    print(already_done_description)
  return False

def create_item(game, *args):
  """Creates an item in some scene."""
  (item, action_desc) = args[0]
  game.curr_location.add_item(item.name, item)
  print(action_desc)
  return False

def talk_to(game, *args):
  """Talk to an NPC/item and get a sentiment analysis."""
  (default_text, sentiment_dict) = args[0]
  dialogue = input("\"")
  blob = TextBlob(dialogue)
  if blob.sentiment.polarity < 0.0:
    if "bad" in sentiment_dict.keys():
      (func, arg) = sentiment_dict["bad"]
      return func(game, arg)
    else:
      print(default_text)
  elif blob.sentiment.polarity == 0.0:
    if "neutral" in sentiment_dict.keys():
      (func, arg) = sentiment_dict["neutral"]
      return func(game, arg)
    else:
      print(default_text)
  else:
    if "good" in sentiment_dict.keys():
      (func, arg) = sentiment_dict["good"]
      return func(game, arg)
    else:
      print(default_text)


def do_more_funcs(game, *args):
  """Does all of these actions"""
  func_arg_list = args[0]
  for fun_arg in func_arg_list:
    (fun, arg) = fun_arg
    fun(game, arg)

def end_game(game, *args):
  """Ends the game."""
  end_message = args[0]
  print(end_message)
  return True

## Game Data

Here's where you can define the locations and items in your game.  To get you started, I defined a super-simple fishing game, which contains the first 3 locations of __Action Castle__ by Jared A. Sorensen, which is part of the awesome book [Parsley](http://www.memento-mori.com/parsely-products/parsely-pdf).  

You can play through the whole game with the following commands:
1. take pole
2. go out
3. south 
4. catch fish with pole
5. eat fish

In [0]:
def build_game():
  # Locations
  cabin = Location("Cabin", "This was your Cabin aboard the spaceship.")
  dark_hallway = Location("Dark Hallway", "This is a dark hallway that leads to other parts of the ship, the lights are dimmed out, and there is an annoying siren blaring about.")
  engine_room = Location("Engine Room", "This is the engine room; it contains the ship's slip space engine that powers the ship; try not to break anything.")
  sketchy_closet = Location("Sketchy Closet", "You are standing in a sketchy closet; some cleaning supplies and bedrolls are sitting around on shelves; it smells musty.")
  messhall = Location("Messhall", "You are in the messhall. Rows of shiny tables and benches are lined up, the crew normally gathers here to eat.")
  rec_center = Location("Rec Center", "You are in the rec center; pool tables and chess tables are scattered about.")
  armory = Location("Armory", "You are in the armory, this is where all the weapons are stored.")
  vault = Location("Vault", "You are in the vault; it apparently contains something pretty valuable.")
  bridge = Location("Bridge", "You are in the bridge; the captain would be piloting the ship here if he were still alive.")
  escape_pod = Location("Escape Pod", "You are in the escape pod, you can finally escape this hellish nightmare. YOU WIN.", end_game=True)
  plasma_cannon = Location("Plasma Cannon", "You are in the plasma cannon; this weapon is used to defend the ship from attacks. Too bad nobody competent is here to use it.")

  # Connections
  cabin.add_connection("south", dark_hallway)
  dark_hallway.add_connection("east", engine_room)
  dark_hallway.add_connection("west", sketchy_closet)
  dark_hallway.add_connection("south", messhall)
  messhall.add_connection("east", rec_center)
  messhall.add_connection("south", armory)
  armory.add_connection("west", vault)
  armory.add_connection("down", plasma_cannon)
  armory.add_connection("east", bridge)
  bridge.add_connection("in", escape_pod)

  # Items that you can pick up
  charge = Item("plasma charge", "a plasma charge", "A PLASMA CHARGE, HIGHLY EXPLOSIVE, WILL PROBABLY COME IN HANDY. DON'T EAT IT", "WORT WORT WORT", start_at=cabin)
  engine = Item("engine", "a slip space engine", "A VERY EXPENSIVE LOOKING ENGINE, IT STRUGGLES TO POWER THE SHIP UNDER THE BARRAGE OF PLASMA FIRE. TRY NOT TO BREAK IT.",  start_at=engine_room, gettable=False)
  powercell = Item("powercell", "a powercell", "A SMALL PORTABLE BATTERY USED TO POWER SOMETHING. IF YOU EAT IT YOU MIGHT GLOW.", start_at=None) 
  matt_recording = Item("recording", "A voice recording of movie star Matt Damon", "A RECORDING DECIDE WITH MATT DAMO'S VOICE. YO UCOULD PROBABLY MAKE A QUICK BUCK BY SELLING THIS ON EBAY.", start_at=None, gettable=False)
  booze = Item("booze", "a cask of booze", "A SIZABLE CASK OF BOOZE ENOUGH TO QUENCH THE SOULS OF A HARDY CREW FOR A WEEK.", "GOOD IDEA, IF YOU'RE GONNA DIE, YOU MIGHT AS WELL BE PLASTERED", start_at=messhall)
  codes = Item("codes", "escape pod codes", "A NOTE WITH THE CODES TO THE ESCAPE PODS. IF YOU FIND ONE YOU COULD GET OFF THIS SHIP", start_at=None)
  gun = Item("gun", "a heavy, machine gun", "A HEAVY MACHINE GUN USED WITH A HIGH RATE OF FIRE AND A BELT OF HOLLOW POINT AMMO. LOOKING AT IT MAKES YOU FEEL LIKE RAMBO.", "SAY HELLO TO MY LITTLE FRIEND!", start_at=armory)
  rock = Item("rock", "a strange, glowing rock", "A ROCK THAT PULSES WITH BLUE LIGHT. MAYBE YOU WOULD KNOW WHAT IT WOULD DO IF YOU ACTUALLY PAID ATTENTION.", start_at=vault)

  #NPCs
  patrons = Item("patrons", "drunk alien patrons", "A BUNCH OF INTOXICATED ALIENS TRYING TO HAVE A GOOD TIME. YOU DON'T REMEMBER ANY OF THEM BEING PART OF THE CREW", start_at=rec_center, gettable=False)
  happy_patrons = Item("patrons", "drunk alien patrons", "A BUNCH OF INTOXICATED ALIENS TRYING TO HAVE A GOOD TIME. YOU MAY NOT KNOW THEM BUT THEY LOVE YOU.", start_at=None, gettable=False)
  matt = Item("MattDamon", "Movie star Matt Damon", "MATT LOOKS A BIT NERVOUS, BUT AFTER GETTING RESCUED SO MANY TIMES FROM PERIL, HE IS PRETTY USE TO THIS KIND OF STUFF.", start_at=sketchy_closet, gettable=False)

  #scenery
  goo = Item("goo", "alien goo", "THIS GOO ONCE MADE UP THE WALL THAT BLOCKED YOUR WAY TO THE ENGINE ROOM. IT'S DEFINITELY DEAD NOW YOU MONSTER.", start_at=None, gettable=False)
  window = Item("window", "a window facing the void", "UPON LOOKING OUT THE WINDOW, YOU SEE A LARGE ALIEN SHIP FIRING PLASMA. YOU BETTER GET OUT OR GET RID OF IT.", start_at=messhall, gettable=False)
  console = Item("console", "a simple set up used to fire the ship's plasma cannons", "A CONSOLE WITH A BIG RED BUTTON AND A STRANGE TUBE TO THE SIDE. YOU MAY NEED TO USE SOMETHING ON THE TUBE.", start_at=plasma_cannon, gettable=False)
  computer = Item("computer", "a complex computer", "AN EXPENSIVE COMPUTER FROM WHICH THE SHIP'S SYSTEMS ARE CONTROLLED.", start_at=bridge, gettable=False)
  shot_computer = Item("computer", "a complex computer filled with holes", "AN EXPENSIVE COMPUTER FROM WHICH THE SHIP'S SYSTEMS ARE CONTROLLED. IT IS NOW FILLED WITH BULLET HOLES THANKS TO YOU.", start_at=None, gettable=False)
  alien_wall = Item("wall", "a weird alien wall", "A VERY STRANGE LIVING WALL, BLOCKING YOUR PATH. IT SEEMS TO PULSE A BIT—GROSS.", start_at=dark_hallway, gettable=False) #blocker
  lock = Item("lock", "the escape pod lock", "A LOCK TO THE ESCAPE POD. WHAT IDIOT WOULD PUT A LOCK ON THIS THING?!", start_at=bridge, gettable=False) #blocker
  door = Item("door", "a sturdy metal door", "A STURDY METAL DOOR, BUILT STURDY FOR SAFETY. HOW IRONIC.", start_at=messhall, gettable=False) #blocker

  #blockers
  dark_hallway.add_block("east", "A weird wall of alien goo blocks the way. You push lightly on the wall but find it's quite solid. And now your hand is slimy.", preconditions={"!location_has_item":alien_wall})
  messhall.add_block("south", "You push with all your might on the door. You're too weak!", preconditions={"!location_has_item":door})
  bridge.add_block("in", "You can't go into the pod. It's locked!", preconditions={"!location_has_item":lock})

  # Add special functions to your items
  charge.add_action("eat plasma charge", end_game, ("You hear a 'faint' beating that gets quicker with time, I guess this is one way to lose weight. YOU LOSE."), preconditions={"inventory_contains":charge})
  charge.add_action("detonate plasma charge", end_game, ("The last thing that goes through you brain is: 'Why am I still holding this?' YOU LOSE."), preconditions={"inventory_contains":charge})
  alien_wall.add_action("destroy wall", describe_something, ("You punch the alien wall, and your hand slips into its fleshy body. It's warm and you definitely felt a couple teeth. You take your hand out a changed person."))
  alien_wall.add_action("lick wall", describe_something, ("You lick the wall, it kinda tastes like raw shrimp and feet... you kind of like it. Weirdo"))
  alien_wall.add_action("destroy wall with plasma charge", 
                        do_more_funcs, ([(destroy_item,  
                                          (charge, "You stick the plasma charge on the wall and start the timer", "You used the plasma, remember?")),
                                          (destroy_item, (alien_wall, "A blinding blue light erupts from the wall, it seems to let out a small squeak before being incinerated in plasma", "")),
                                          (create_item, (goo, "A small pile of goo is all that remains"))
                                          ]), 
                        preconditions={"inventory_contains":charge, "location_has_item":alien_wall})
  engine.add_action("fix engine", end_game, ("As the engine explodes, it becomes abundantly clear that you don't know what you are doing. YOU LOSE."))
  engine.add_action("kick engine", add_item_to_inventory, (powercell, "You decide to kick this million dollar testmant to human engineering... it churns to life, and spits out a powercell into your hand.", "You already have a powercell"))
  engine.add_action("eat engine", describe_something, ("Despite your best efforts, you're stomach can't contain this several ton slip space engine, you gluttnous fool"))
  matt.add_action("greet him", describe_something, ("You salute matt and he says to you, 'Do not go gently into that good night.' Inredible!"))
  matt.add_action("rescue him", add_item_to_inventory, (matt_recording, "You tell Matt that you are here to rescue him; he stares longinly into your eyes and says, 'Rage rage against the dying light.' and hands you a voice recording.", "He already gave you the recording"))
  matt.add_action("eat him", end_game, ("You try to bite Matt Damon, but he grabs your head, and whispers in your ear, 'Jesus Christ it's Jason Bourne'. And then you die from his mere presence. YOU LOSE."))
  matt.add_action("kiss him", describe_something, ("You lean in for a kiss, but Matt refuses as he has been happily married for 15 years. Guess you're just another crazy fan"))
  matt.add_action("marry him", describe_something, ("Matt Damon does not want to marry you"))
  matt.add_action("open a comedy club with him", describe_something, ("You guys choose a small spot in brooklyn, business starts off slow, but hey you're you and he's Matt Damon, and you guys know it's gonna work out. One night, on a candid summer day, you get an offer you can't refuse. Comedy legend Dave Chapelle wants to shoot a special at your club, and wait a second... you're still on a space ship, fending for your life. Get back on topic!"))
  
  matt.add_action("talk to him", talk_to, ("Matt Damon looks at you weirdly.",
                                           {"bad": (end_game, ("Matt Damon does not appreciate what you said. He's so offended, he decides to kill you."))}))

  powercell.add_action("step on powercell", end_game, ("You throw the powercell on the ground and stomp down. Electricity surges through your body and stops your heart. Nice going. YOU LOSE."), preconditions={"inventory_contains":powercell})
  matt_recording.add_action("listen to recording", describe_something, ("You hear Matt's wonderful voice say, 'Hello, I, movie star Matt Damon, need rescuing. You know what to do"))
  door.add_action("open door", describe_something, ("You try your hardest but your tiny chicken leg fingers can't pry open this metal door"))
  door.add_action("open door with powercell", do_more_funcs, 
                                        ([(destroy_item, (powercell, "You look at the door, and then look at the powercell in your hand. You step back and chuck the powercell at the door...", "You used the powercell, remember?")),
                                          (destroy_item, (door, "The door flies open. For some reason... How did that work? You think.", "")),
                                          ]), 
                        preconditions={"inventory_contains":powercell, "location_has_item":door})
  window.add_action("lick window", describe_something, ("Well, it's the same window, now just with some idiots saliva on it."))   
  booze.add_action("drink booze", end_game, ("You plop down on the ground and start downing booze, thinking about your ex, Kristen, who stole your pet goldfish. You manage to die of alcohol poisoning. YOU LOSE."), preconditions={"inventory_contains":booze}) 
  booze.add_action("smell booze", describe_something, ("You smell a rich cornucopia of flavors from all around Earth. Very tempting."), preconditions={"inventory_contains":booze})
  patrons.add_action("talk to patrons", describe_something, ("You start walking up to the patrons, but the music stops, and they all stare daggers at you. Before you can say anything, they fill you with bullets. The last thing you remember is the music start back up, and a waitress with a tray steps over you. Guess you weren't cool enough."))
  patrons.add_action("yell at patrons", describe_something, ("You start walking up to the patrons, but the music stops, and they all stare daggers at you. Before you can say anything, they fill you with bullets. The last thing you remember is the music start back up, and a waitress with a tray steps over you. Guess you weren't cool enough."))
  patrons.add_action("dance with patrons", describe_something, ("You start walking up to the patrons, but the music stops, and they all stare daggers at you. Before you can say anything, they fill you with bullets. The last thing you remember is the music start back up, and a waitress with a tray steps over you. Guess you weren't cool enough."))
  patrons.add_action("drink with patrons", describe_something, ("You start walking up to the patrons, but the music stops, and they all stare daggers at you. Before you can say anything, they fill you with bullets. The last thing you remember is the music start back up, and a waitress with a tray steps over you. Guess you weren't cool enough."))
  patrons.add_action("give booze to patrons", do_more_funcs, 
                                        ([(destroy_item, (booze, "You raise your cask of booze and exclaim, 'DRINKS ARE ON ME!'", "You used the booze, remember?")),
                                          (destroy_item, (patrons, "The patrons cheer for you. And bring you in to the mix. Too bad you aren't this good at making friends in real life.", "")),
                                          (create_item, (happy_patrons, "You don't remember much from the party, but all the patrons love you now.")),
                                          (add_item_to_inventory, (codes, "You somehow wound up with the codes to the escape pod", ""))]), 
                        preconditions={"inventory_contains":booze, "location_has_item":patrons})
  patrons.add_action("talk to them", talk_to, ("Matt Damon looks at you weirdly.",
                                           {"bad": (end_game, ("Matt Damon does not appreciate what you said. He's so offended, he decides to kill you."))}))
  happy_patrons.add_action("party with patrons", describe_something, ("The patrons greet you cordially, but shortly into partying, you realize there's more important things to do."))
  gun.add_action("shoot gun", describe_something, ("You pull the trigger, and immediately feel an intense kick against your shoulder, and a stacato of bullets fly through the barrel. That was quite fun!"), preconditions={"inventory_contains":gun})
  gun.add_action("lick gun", describe_something, ("Come on man, other people use that..."), preconditions={"inventory_contains":gun})
  gun.add_action("shoot yourself with gun", end_game, ("You shoot yourself with the gun. Well, if that's what you wanted. You had so much to live for. :^( YOU LOSE."), preconditions={"inventory_contains":gun})
  rock.add_action("lick rock", describe_something, ("It kind of tastes like candy..."), preconditions={"inventory_contains":rock})
  rock.add_action("eat rock", end_game, ("Somehow, you manage to crunch the rock. When you swallow your first gravelly bunch, you feel your stomach melting. Oh dear, your stomach explodes. YOU LOSE."), preconditions={"inventory_contains":rock})
  console.add_action("use console", end_game, ("You position the firing console toward the enemy ship. You press the button to fire. The console was not charged and only gave away your position. The alien ship's guns now train on your ship. You enjoyed the moment as much as you could before they incinerated you. YOU LOSE."))
  console.add_action("use console with glowing rock", end_game, ("You decide to put the glowing rock in the tube. After a brief second, the button begins glowing red, and all the lights seem a bit brighter. You hit the button, and with the fury of a thousand suns, the hostile ship is vaporized in an instance. You defenitely meant to do that. YOU WIN."), preconditions={"inventory_contains":rock})
  computer.add_action("punch computer", describe_something, ("You punch the computer, good job! you need anger management classes."))
  computer.add_action("use computer", describe_something, ("Despite your best efforts, you can't figure out how to bypass the computer security. It's a feature not a bug..."))
  computer.add_action("shoot computer", do_more_funcs, 
                                        ([(destroy_item, (computer, "You raise your gun and scream, putting hundreds of bullets into the computer in a fit of rage.", "")),
                                          (create_item, (shot_computer, "After a bit, you stop and hear the computer say, 'Access granted, Welcome back captain'"))]),
                        preconditions={"inventory_contains":gun, "location_has_item":computer})
  shot_computer.add_action("call for help with computer", describe_something, ("You radio for help, but no one picks up. Guess you're not important enough"))
  shot_computer.add_action("use recording to call for help", end_game, ("You play Matt Damon's recording into the computer, and almost immediately, an armada of ships jumps into the vicinity. They decimate the hostile vessel, saving you, but most importantly, rescuing Matt Damon. Nobody messes with Matt Damon and gets away with it. YOU WIN."), preconditions={"inventory_contains":matt_recording})
  lock.add_action("open lock with code", destroy_item, (lock, "You punch in the code, and the lock opens. Granting you access to the escape pod", ""), preconditions={"inventory_contains":codes, "location_has_item":lock})
  lock.add_action("eat lock", describe_something, ("Yeah that does it, you bite down on the lock and it opens allowing you to... just kidding. You break your tooth and you are immense pain."))
  
  g = Game(cabin)
  return g


# Play the game
This small snippet of code is what you need to run the game.  Behold! The magestic prompt! 

In [0]:
import nltk
nltk.download('punkt')
from textblob import TextBlob

def game_loop():
  game = build_game()
  parser = Parser(game)
  game.describe()

  command = ""
  while not (command.lower() == "exit" or command.lower == "q"):
    command = input(">")
    end_game = parser.parse_command(command)
    if end_game:
      return

game_loop()
print('THE GAME HAS ENDED.')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
This was your Cabin aboard the spaceship.
Exits: South
You see: 
a plasma charge
	 eat plasma charge
	 detonate plasma charge
>s
This is a dark hallway that leads to other parts of the ship, the lights are dimmed out, and there is an annoying siren blaring about.
Exits: North, East, West, South
You see: 
a weird alien wall
	 destroy wall
	 lick wall
	 destroy wall with plasma charge
>w
You are standing in a sketchy closet; some cleaning supplies and bedrolls are sitting around on shelves; it smells musty.
Exits: East
You see: 
Movie star Matt Damon
	 greet him
	 rescue him
	 eat him
	 kiss him
	 marry him
	 open a comedy club with him
	 talk to him
>talk to him
"hi
Matt Damon looks at you weirdly.
>sup
I'm not sure what you want to do.


# Visualize your game
The code below allows you to create a directed graph that shows the locations in your game and how they are connected.  You can also save a PDF of your graph to your Google Drive with the `save_to_drive` method.  The output file will be called `game-visualization.pdf`.

In [0]:
#!pip install graphviz
from graphviz import Digraph
from IPython.display import Image
import queue

def DFS(game, graph):
  """Do a depth-first-search traversal of the locations in the game
     starting at the start location, and create a GraphViz graph 
     to vizualize the connections between the locations, and the items
     that are located at each location."""
  start_location = game.curr_location
  frontier = queue.Queue()
  frontier.put(start_location)
  visited = {}
  visited[start_location.name] = True

  while not frontier.empty():
    current_location = frontier.get()
    game.curr_location = current_location
    name = current_location.name
    description = current_location.description
    items = current_location.items
    items_html = describe_items(current_location)
    html = "<<b>%s</b><br />%s<br />%s>" % (name, description, items_html)
    # Create a new node in the graph for this location
    graph.node(name, label=html)  

    connections = current_location.connections
    for direction in connections.keys():
      next_location = connections[direction]
      if not current_location.is_blocked(direction, game):
        # Create an edge between the current location and its successor
        graph.edge(name, next_location.name, label=direction.capitalize())
      else:
        # Create a dotted edge for connected locations that are blocked
        block_description = "%s\n%s" % (direction.capitalize(), current_location.get_block_description(direction))
        graph.edge(name, next_location.name, label=block_description, style="dotted")
      if not next_location.name in visited:
        visited[next_location.name] = True
        frontier.put(next_location)

def describe_items(location, print_commands=True):
    """Describe what objects are in the current location."""
    items_html = ""
    if len(location.items.keys()) > 0:
      items_html = "You see: "
    for item_name in location.items:
      item = location.items[item_name]
      items_html += item.description
      if print_commands:
        special_commands = item.get_commands()
        for cmd in special_commands:
          items_html += "<br/><i>%s</i>" % cmd
    return items_html

def save_to_drive(graph):
  from google.colab import drive
  drive.mount('/content/drive/')
  graph.render('/content/drive/My Drive/game-visualization', view=True)  

graph = Digraph(node_attr={'color': 'lightblue2', 'style': 'filled'})
game = build_game()
DFS(game, graph)
#save_to_drive(graph)
graph
